
# Load Data, Drop Unecessary Information

In [ ]:
import pandas as pd
import os

In [ ]:
def create_complete_df(num_files):
    path_to_features  = '/content/drive/MyDrive/SOCData'

    list_of_dfs = []
    for i in range(num_files):
        csv_file = f'{i}.csv'
        print(csv_file)
        df = pd.read_csv(os.path.join(path_to_features, csv_file))
        list_of_dfs.append(df)

    df = pd.concat(list_of_dfs, axis=0)
    return df

In [ ]:
df = create_complete_df(28)

In [ ]:
df

In [ ]:
df.reset_index(inplace = True)

In [ ]:
df.drop(['system:index', '.geo'], axis = 1, inplace = True)

In [ ]:
df

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(verbose=2, max_iter=20)

imputed_df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
imputed_df

# Merge HWSD Data

In [ ]:
HWSD = pd.read_csv('/content/drive/MyDrive/SOCData/HWSD_DATA.csv')
HWSD = HWSD[['MU_GLOBAL', 'S_OC']]
HWSD

In [ ]:
new = pd.merge(imputed_df, HWSD, on="MU_GLOBAL")
new = new.dropna()

# Split Data

In [ ]:
y = new.pop('S_OC')
X = new

In [ ]:
assert 'S_OC' not in X.columns

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

In [ ]:
assert len(X_train) == len(y_train)

In [ ]:
assert len(X_test) == len(y_test)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X, y)

# RMSE

In [ ]:
from sklearn.metrics import mean_squared_log_error

round(mean_squared_log_error(rf.predict(X_test), y_test), 3)